<a href="https://colab.research.google.com/github/rollways/JD_EDA/blob/main/item2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O taipei_sans_tc_beta.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
!mv taipei_sans_tc_beta.ttf /usr/local/lib/python3.7/dist-packages/matplotlib//mpl-data/fonts/ttf

from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt 

# 自定義字體變數
myfont = FontProperties(fname=r'/usr/local/lib/python3.7/dist-packages/matplotlib/mpl-data/fonts/ttf/taipei_sans_tc_beta.ttf')

# !!!!後續在相關函式中增加fontproperties=myfont屬性即可!!!!

--2022-01-04 05:35:43--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 142.250.81.206, 2607:f8b0:4004:82f::200e
Connecting to drive.google.com (drive.google.com)|142.250.81.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/51l4luq30pj392kbn1u8cbdqph7dnrar/1641274500000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_ [following]
--2022-01-04 05:35:44--  https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/51l4luq30pj392kbn1u8cbdqph7dnrar/1641274500000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)... 142.250.81.193, 2607:f8b0:4004:82f::2001
Connecting to doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)|142.250.81.193|

### load Data

In [14]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format',lambda x:'%.0f'%x)


data = pd.read_csv('/content/drive/MyDrive/T大使/data3_project.csv')
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,member_start_date,first_trans_date
0,PS5327001158,JA01,晶淬雪潔顏乳100ml,8,2019-01-01,1259,9100002488768,中區,2019-12-09,2018-06-24
1,PS5335020955,KG06,多肽亮眼乳霜20g-保濕型,8,2019-01-01,839,9100002556566,中區,2018-10-21,2018-10-21
2,PS5335020955,BJ102,檸檬精油10ml MUI,8,2019-01-01,839,9100002556566,中區,2018-10-21,2018-10-21
3,PS5335020955,JZA3G1,柚見快樂精油15ml MUI,-8,2019-01-01,-1501,9100002556566,中區,2018-10-21,2018-10-21
4,PS5444006919,BJ103,細花冬青香氛油10ml MUI,8,2019-01-01,672,9100002582077,南區,2018-12-04,2018-12-04
...,...,...,...,...,...,...,...,...,...,...
1404430,PS5324024941,KQ02,玫瑰晶萃活膚液130ml,16,2020-12-31,2946,9100000595826,中區,2010-03-15,2012-07-11
1404431,PS5219002027,GAXFG1,真正除臭抗菌噴霧(銀離子迷迭香)250ml,8,2020-12-31,309,9100003028369,竹區,2020-12-08,2020-11-26
1404432,PS5219002027,MAB2G1,BA-5肌密全效噴霧奇蹟水70ml,8,2020-12-31,1094,9100003028369,竹區,2020-12-08,2020-11-26
1404433,PS5219002027,WD096,緊緻保濕裸膚羽翼面膜25ml-零售單包,16,2020-12-31,165,9100003028369,竹區,2020-12-08,2020-11-26


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404435 entries, 0 to 1404434
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   InvoiceNo          1404435 non-null  object 
 1   StockCode          1404435 non-null  object 
 2   Description        1404435 non-null  object 
 3   Quantity           1404435 non-null  float64
 4   InvoiceDate        1404435 non-null  object 
 5   UnitPrice          1404435 non-null  float64
 6   CustomerID         1403724 non-null  float64
 7   Country            1404435 non-null  object 
 8   member_start_date  1366651 non-null  object 
 9   first_trans_date   1400412 non-null  object 
dtypes: float64(3), object(7)
memory usage: 107.2+ MB


In [16]:
data.isnull().any()

InvoiceNo            False
StockCode            False
Description          False
Quantity             False
InvoiceDate          False
UnitPrice            False
CustomerID            True
Country              False
member_start_date     True
first_trans_date      True
dtype: bool

In [19]:
data.drop(data.loc[data['CustomerID'].isna()].index,inplace=True)

In [20]:
data.isnull().any()

InvoiceNo            False
StockCode            False
Description          False
Quantity             False
InvoiceDate          False
UnitPrice            False
CustomerID           False
Country              False
member_start_date     True
first_trans_date      True
dtype: bool

In [21]:
#title
#聚合得到userID,itemID列表
df_group = data.groupby(['CustomerID'])['Description'].apply(lambda x:''.join([str(m) for m in x])).reset_index()
df_group.head()

,CustomerID,Description
0,9100000000061,24H賦活液100ml玫瑰晶萃瞬效霜30ml超防禦UV潤色妝前乳30ml旗艦無壓力頭SPA課...
1,9100000000085,五日激光煥顏課程單堂膠原晶球水導膜課程單堂苦杏仁酸靚顏慕絲300ml旗艦全背順氣活力SPA課...
2,9100000000092,超導水網瞬效面膜26ml*6包24小時黃金璀璨賦活液40ml玫瑰晶萃露15ml玫瑰晶萃活膚液...
3,9100000000115,玫瑰晶萃水面膜10ml*10支DC原生龍血素30ml阿甘髮膜1000ml阿甘洗髮精1000m...
4,9100000000170,旗艦全背順氣活力SPA課程2.0單堂世界花園旅行組(黑)


In [22]:
df_group.to_csv("/content/drive/MyDrive/T大使/itemlens_uid_itemids.csv",index=False)

### Use Pyspark train item2vec

In [23]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u292-b10-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u292-b10-0ubuntu1~18.04_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [24]:
!pip install pyspark
!pip install findspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 49.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=851dece762125f0f8890d807a1d8b15cc12509f1225ba4e099ec72d28170539f
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [25]:
#初始化spark
import findspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark Item2vec").getOrCreate()

sc = spark.sparkContext

In [26]:
df = spark.read.csv("/content/drive/MyDrive/T大使/itemlens_uid_itemids.csv",header=True)
df.show(5)

+---------------+-------------------------------------+
|     CustomerID|                          Description|
+---------------+-------------------------------------+
|9100000000061.0|        24H賦活液100ml玫瑰晶萃瞬效...|
|9100000000085.0|五日激光煥顏課程單堂膠原晶球水導膜...|
|9100000000092.0|        超導水網瞬效面膜26ml*6包24...|
|9100000000115.0|         玫瑰晶萃水面膜10ml*10支DC...|
|9100000000170.0|      旗艦全背順氣活力SPA課程2.0單...|
+---------------+-------------------------------------+
only showing top 5 rows



In [28]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

#把字符串格式變成LIST形式
df = df.withColumn('Description', F.split(df.Description,""))

### 實現word(item)2vec的訓練與轉換

In [30]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(
    vectorSize=5,
    minCount=0,
    inputCol="Description",
    outputCol="Stock_2vec")

model = word2Vec.fit(df)

In [31]:
#不計算每個user的embedding,而是計算item的embedding
model.getVectors().show(3, truncate=False)

+----+-----------------------------------------------------------------------------------------------------+
|word|vector                                                                                               |
+----+-----------------------------------------------------------------------------------------------------+
|E   |[0.31339240074157715,0.11344217509031296,-0.4995526969432831,-1.221623182296753,-0.05249305069446564]|
|色  |[0.32201236486434937,0.5203765034675598,-0.22991943359375,-0.49561482667922974,0.9676986336708069]   |
|底  |[0.35202357172966003,-0.37923166155815125,0.4676963984966278,-0.3361891210079193,1.467858910560608]  |
+----+-----------------------------------------------------------------------------------------------------+
only showing top 3 rows



/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [32]:
model.getVectors().select("Word","vector").toPandas().to_csv('/content/drive/MyDrive/T大使/stocklens_stock_embedding.csv',index=False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


###使用T-SNE視覺化

In [33]:
model.getVectors().show(truncate=False)

+----+-------------------------------------------------------------------------------------------------------+
|word|vector                                                                                                 |
+----+-------------------------------------------------------------------------------------------------------+
|E   |[0.31339240074157715,0.11344217509031296,-0.4995526969432831,-1.221623182296753,-0.05249305069446564]  |
|色  |[0.32201236486434937,0.5203765034675598,-0.22991943359375,-0.49561482667922974,0.9676986336708069]     |
|底  |[0.35202357172966003,-0.37923166155815125,0.4676963984966278,-0.3361891210079193,1.467858910560608]    |
|員  |[0.603043794631958,0.3718326985836029,1.086342215538025,0.03688478097319603,0.7550172805786133]        |
|效  |[1.0869953632354736,0.5731258988380432,-0.19838136434555054,0.23776568472385406,0.45813077688217163]   |
|摩  |[-0.7207176089286804,2.16391658782959,-1.568183183670044,-0.5077064633369446,1.5221469402313232]       |
|璀  |[

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [34]:
word = model.getVectors().select("word")
word = np.array(word.select('word').collect())
word = word.reshape(-1)

item_V = model.getVectors().select("Vector")
item_V = np.array(item_V.select('vector').collect())

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [35]:
word.shape

(812,)